# Data Preperation

In [2]:
from scipy import ndimage
import numpy as np
np.random.seed(123)

In [23]:
import pandas as pd

In [24]:
# tmp_array = ndimage.imread('F:/cifar10/data/train/2.png')

# tmp_array.shape

In [25]:
def convert_images_to_array(path, end, start = 0):
    lst_train = []
#     train_path = 'F:/cifar10/data/train/'
    for i in range(start, end):
        val = (str(i+1))
    #     print(train_path + val+'.png')
        lst_train.append(ndimage.imread(path + val+'.png'))

    #Convert list to nd array    
    return(np.array(lst_train))

In [26]:
# convert train data to ndarray
train_path = 'F:/cifar10/data/train/'
np_train = convert_images_to_array(train_path, 50000)

In [27]:
print(np_train.shape)

(50000, 32, 32, 3)


In [7]:
# Normalize data
np_train = np_train.astype('float32')
np_train = np_train/255.0

## creating labels

In [8]:
df_label = pd.read_csv('F:/cifar10/data/trainLabels.csv')

In [9]:
df_label.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [10]:
## Create a dictionary with unique labels assigned number
lab = list(df_label['label'].unique())
lab_dic = {}
for i in range(len(lab)):
#     print(i)
#     print(lab[i])
    lab_dic[lab[i]] = i
lab_dic

{'airplane': 9,
 'automobile': 3,
 'bird': 4,
 'cat': 7,
 'deer': 2,
 'dog': 8,
 'frog': 0,
 'horse': 5,
 'ship': 6,
 'truck': 1}

In [11]:
a = 'cat'
lab_dic.get(a)

7

In [12]:
# [lab_dic.get(i) for i in df_label['label'] ]

In [13]:
# df_label['label'].unique()

In [14]:
np_label = np.array([lab_dic.get(i) for i in df_label['label'] ])

In [15]:
np_label.shape

(50000,)

In [16]:
import keras

Using TensorFlow backend.


In [17]:
y_nplabel = keras.utils.to_categorical(np_label)

In [18]:
num_classes = y_nplabel.shape[1]
num_classes

10

# Deploy CNN

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [20]:
epochs = 50

In [21]:
# # Create the model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))
# # Compile model
# epochs = 25
# lrate = 0.01
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# print(model.summary())

In [22]:
def simplex_model():
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(32, (3,3), input_shape = (32,32,3), activation='relu'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(1, 1)))

    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

#     epochs = 20
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    
    return model

In [23]:
def simple_model():
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(32, (3,3), input_shape = (32,32,3), activation='relu'))

    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))


    
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    
    return model


In [24]:
model = simple_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 30, 1)         9248      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 30, 1)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 30, 1)         9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 15, 1)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 480)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               246272    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

In [25]:
# Fit the model
model.fit(np_train, y_nplabel, validation_split=0.3, epochs=epochs, batch_size=32)

Train on 35000 samples, validate on 15000 samples
Epoch 1/50
35000/35000 [==============================] - 34s - loss: 1.9435 - acc: 0.2878 - val_loss: 1.7932 - val_acc: 0.3641
Epoch 2/50
35000/35000 [==============================] - 37s - loss: 1.7580 - acc: 0.3622 - val_loss: 1.7125 - val_acc: 0.3857
Epoch 3/50
35000/35000 [==============================] - 39s - loss: 1.6762 - acc: 0.3953 - val_loss: 1.6341 - val_acc: 0.4151
Epoch 4/50
35000/35000 [==============================] - 40s - loss: 1.6124 - acc: 0.4202 - val_loss: 1.5829 - val_acc: 0.4290
Epoch 5/50
35000/35000 [==============================] - 41s - loss: 1.5646 - acc: 0.4385 - val_loss: 1.5237 - val_acc: 0.4551
Epoch 6/50
35000/35000 [==============================] - 40s - loss: 1.5283 - acc: 0.4502 - val_loss: 1.5889 - val_acc: 0.4342
Epoch 7/50
35000/35000 [==============================] - 40s - loss: 1.4888 - acc: 0.4658 - val_loss: 1.5066 - val_acc: 0.4651
Epoch 8/50
35000/35000 [==============================

In [26]:
model.save('F:/cifar10/model/my_simple_model_50.h5')

## Prediction

In [1]:
## load save model
from keras.models import load_model

Using TensorFlow backend.


In [2]:
my_model = load_model('F:/cifar10/model/my_simple_model_50.h5')

In [6]:
# convert test data to ndarray
test_path = 'F:/cifar10/data/test/'
np_test = convert_images_to_array(test_path, 300000)

In [7]:
# print('train data shape', np_train.shape)
print('test data shape', np_test.shape)

test data shape (300000, 32, 32, 3)


In [ ]:
# Normalize data
np_test = np_test.astype('float32')
np_test = np_test/255.0

In [26]:

start = 0
end = 17
batch_size=3

# convert test data to ndarray
test_path = 'F:/cifar10/data/test/'
# np_test = convert_images_to_array(test_path, 300000)

tmp_lst = []

# for i in xrange(0, len(worklist), batch_size):
for i in np.arange(start, end, batch_size):
    if(i + batch_size > end):
        if(i + batch_size < end):
            pass
        else:
#             print(i, end)
            tmp_lst.append(tuple((i, end)))
    else:
#         print(i, i + batch_size)
        tmp_lst.append(tuple((i, i + batch_size)))

for i, j in tmp_lst:
    print(i)
    print(j)

0
3
3
6
6
9
9
12
12
15
15
17


In [ ]:
def get_batch_submission(model, data, batch_size):
    np_prediction = model.predict_classes(data)

    lst_prd = []
    for j in np_prediction:
        for i in lab_dic.items():
            if(i[1] == j):
                lst_prd.append(i[0])
    #             print(i)
    #             print(i[1])
    #             print(i[0])
    lst_prd


    df_submission = pd.DataFrame()
    df_submission['id'] = list(range(1,len(np_prediction)+1))
    df_submission['label'] = lst_prd

    return df_submission

In [ ]:
def get_submission(model, data):
    np_prediction = model.predict_classes(data)

    lst_prd = []
    for j in np_prediction:
        for i in lab_dic.items():
            if(i[1] == j):
                lst_prd.append(i[0])
    #             print(i)
    #             print(i[1])
    #             print(i[0])
    lst_prd


    df_submission = pd.DataFrame()
    df_submission['id'] = list(range(1,len(np_prediction)+1))
    df_submission['label'] = lst_prd

    return df_submission

In [ ]:
df_submission = get_submission(my_model,np_test)

In [ ]:
df_submission.to_csv('F:/cifar10/data/submission.csv', index=False)

### Visualize the data

In [ ]:
from matplotlib import pyplot
from scipy.misc import toimage

In [ ]:
pyplot.imshow(toimage(X_train[3]))
pyplot.show()